# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [25]:
import pandas as pd
import pymongo

In [26]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [125]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


## Question 0

In [23]:
df_youtube.columns[[3,4,2,6]]

Index(['category_id', 'tags', 'channel_title', 'likes'], dtype='object')

### Netoyer les données

In [30]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])
 

df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
 
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
 
l_d =df_youtube.to_dict(orient='records')
 
import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
 
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [32]:
from pymongo import MongoClient

client = MongoClient("mongo:27017")

bdd_yt = client.yt

bellifa_collection_yt = bdd_yt['videos']

bellifa_collection_yt.delete_many({})

bellifa_collection_yt.insert_many(l_d)

## Question 1  

In [36]:
request = bellifa_collection_yt.find({'channel_title':'Apple'})
request.count()
list(request)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Science & Technology'},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skat

## Question 2

In [104]:
request = bellifa_collection_yt.aggregate([{ "$group": { "_id": "$category_id"}  },{ "$group": { "_id": "category_id", "count": { "$sum": 1 } } } ])
list(request)

[{'_id': 'category_id', 'count': 16}]

## Question 3

## Question 4

In [175]:
request = bellifa_collection_yt.aggregate([{"$group" : {"_id" : "$_id", "lesPlusVu" : {"$max" : "$views"}}}
                                     , { "$sort" : { "lesPlusVu" : -1 } }     ])
list(request)


[{'_id': 'MBdVXkSdhwU', 'lesPlusVu': 41500672},
 {'_id': 'tt2k8PGm-TI', 'lesPlusVu': 36323498},
 {'_id': 'LunHybOKIjU', 'lesPlusVu': 33191594},
 {'_id': 'SHq2qrFUlGY', 'lesPlusVu': 32136948},
 {'_id': 'Q0CbN8sfihY', 'lesPlusVu': 27909589},
 {'_id': 'K4wEI5zhHB0', 'lesPlusVu': 19707391},
 {'_id': 'r9-DM9uBtVI', 'lesPlusVu': 19194567},
 {'_id': '3y5A4paFOb4', 'lesPlusVu': 18809766},
 {'_id': 'GO6qs83CHpc', 'lesPlusVu': 15952413},
 {'_id': 'D59v74k5flU', 'lesPlusVu': 15579127},
 {'_id': 'FSOxXsYVJMQ', 'lesPlusVu': 15035822},
 {'_id': '5ggZ9jIHnr8', 'lesPlusVu': 14849524},
 {'_id': 'J_ub7Etch2U', 'lesPlusVu': 14077967},
 {'_id': 'dA5qYrboTUE', 'lesPlusVu': 13752002},
 {'_id': 't2oVUxTV4WA', 'lesPlusVu': 13740234},
 {'_id': 'AX8-YzMKZhQ', 'lesPlusVu': 13728070},
 {'_id': 'LcZ2AuvxXNA', 'lesPlusVu': 13686054},
 {'_id': 'APHgDFRpCi0', 'lesPlusVu': 13077467},
 {'_id': '1NyMSWqIJDQ', 'lesPlusVu': 12115465},
 {'_id': 'gOsM-DYAEhY', 'lesPlusVu': 11985474},
 {'_id': 'EUoe7cf0HYw', 'lesPlusVu': 107

## Question 5

In [173]:
request = bellifa_collection_yt.aggregate([
        {"$group" : {"_id" : "$_id", "moyenne de vu par categorie" : {"$avg" : "$views"}}}
         , { "$sort" : { "moyenne de vu par categorie" : -1 } } 
    ])
list(request)

[{'_id': 'MBdVXkSdhwU', 'moyenne de vu par categorie': 41500672.0},
 {'_id': 'tt2k8PGm-TI', 'moyenne de vu par categorie': 36323498.0},
 {'_id': 'LunHybOKIjU', 'moyenne de vu par categorie': 33191594.0},
 {'_id': 'SHq2qrFUlGY', 'moyenne de vu par categorie': 32136948.0},
 {'_id': 'Q0CbN8sfihY', 'moyenne de vu par categorie': 27909589.0},
 {'_id': 'K4wEI5zhHB0', 'moyenne de vu par categorie': 19707391.0},
 {'_id': 'r9-DM9uBtVI', 'moyenne de vu par categorie': 19194567.0},
 {'_id': '3y5A4paFOb4', 'moyenne de vu par categorie': 18809766.0},
 {'_id': 'GO6qs83CHpc', 'moyenne de vu par categorie': 15952413.0},
 {'_id': 'D59v74k5flU', 'moyenne de vu par categorie': 15579127.0},
 {'_id': 'FSOxXsYVJMQ', 'moyenne de vu par categorie': 15035822.0},
 {'_id': '5ggZ9jIHnr8', 'moyenne de vu par categorie': 14849524.0},
 {'_id': 'J_ub7Etch2U', 'moyenne de vu par categorie': 14077967.0},
 {'_id': 'dA5qYrboTUE', 'moyenne de vu par categorie': 13752002.0},
 {'_id': 't2oVUxTV4WA', 'moyenne de vu par categ

## Question 6 

In [174]:
request = bellifa_collection_yt.aggregate([
        {"$group" : {"_id" : "$_id", "maxmoyenne" : {"$avg" : "$likes"}}}
             , { "$sort" : { "maxmoyenne" : -1 } } 

    ])
list(request)

[{'_id': 'MBdVXkSdhwU', 'maxmoyenne': 2010366.0},
 {'_id': 'tt2k8PGm-TI', 'maxmoyenne': 1431683.0},
 {'_id': '3y5A4paFOb4', 'maxmoyenne': 1075133.0},
 {'_id': 'FSOxXsYVJMQ', 'maxmoyenne': 1056548.0},
 {'_id': 'LunHybOKIjU', 'maxmoyenne': 982842.0},
 {'_id': '04UjShXZbD0', 'maxmoyenne': 921577.0},
 {'_id': 'ktc8XDBq93k', 'maxmoyenne': 793387.0},
 {'_id': 'cLdxuaxaQwc', 'maxmoyenne': 706104.0},
 {'_id': 'y4D1KtVetKM', 'maxmoyenne': 656526.0},
 {'_id': 'Q0CbN8sfihY', 'maxmoyenne': 631087.0},
 {'_id': 'Tm8LGxTLtQk', 'maxmoyenne': 624601.0},
 {'_id': '5ggZ9jIHnr8', 'maxmoyenne': 618436.0},
 {'_id': 'gOsM-DYAEhY', 'maxmoyenne': 611911.0},
 {'_id': 'SHq2qrFUlGY', 'maxmoyenne': 578266.0},
 {'_id': 'EUoe7cf0HYw', 'maxmoyenne': 508015.0},
 {'_id': 'CwfoyVa980U', 'maxmoyenne': 499205.0},
 {'_id': 'AX8-YzMKZhQ', 'maxmoyenne': 468636.0},
 {'_id': 'iKzRIweSBLA', 'maxmoyenne': 461378.0},
 {'_id': 'J_ub7Etch2U', 'maxmoyenne': 452820.0},
 {'_id': 'LcZ2AuvxXNA', 'maxmoyenne': 432783.0},
 {'_id': '03IUgO